In [1]:
import os

from utils_park import *
from vocab import *

from tokenizers import Tokenizer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
dir = '../newsbt_data/'
out_path = os.path.join(dir, '40000_vocab_park/')

PAD = '<pad>'
UNK = '<unk>'
SOS = '<start>'
EOS = '<end>'

PAD_ID = 0
UNK_ID = 1
SOS_ID = 2
EOS_ID = 3

train_code_path = os.path.join(dir, 'train/train.token.code')
train_type_path = os.path.join(dir,'train/train.token.simsbt')

valid_code_path = os.path.join(dir, 'valid/valid.token.code')
valid_type_path = os.path.join(dir,'valid/valid.token.simsbt')

test_code_path = os.path.join(dir, 'test/test.token.code')
test_type_path = os.path.join(dir,'test/test.token.simsbt')

In [3]:
train_code = read_file(train_code_path)
test_code = read_file(test_code_path)
valid_code = read_file(valid_code_path)

train_type = read_file(train_type_path)
test_type = read_file(test_type_path)
valid_type = read_file(valid_type_path)

In [4]:
split_train_code = split_code(train_code)
split_test_code = split_code(test_code)
split_valid_code = split_code(valid_code)

In [5]:
def join_code_type_withtoken(code_data, type_data):
    data = []
    for i, j in zip(code_data, type_data):
        temp = i + ' <code> ' + j
        data.append(temp)
    return data


In [6]:
train_data4 = join_code_type_withtoken(split_train_code, train_type)
test_data4 = join_code_type_withtoken(split_test_code, test_type)
valid_data4 = join_code_type_withtoken(split_valid_code, valid_type)

In [7]:
# out path
train_out_path = '../newsbt_data/train/'
test_out_path = '../newsbt_data/test/'
valid_out_path = '../newsbt_data/valid/'

In [8]:
save_file(train_out_path, "code_simsbt_camel", train_data4)
save_file(test_out_path, "code_simsbt_camel", test_data4)
save_file(valid_out_path, "code_simsbt_camel", valid_data4)

# Make Vocab

In [9]:
train_ccode_simsbt_camel_lines = read_file('../newsbt_data/train/code_simsbt_camel')
test_code_simsbt_camel_lines = read_file('../newsbt_data/test/code_simsbt_camel')
valid_code_simsbt_camel_lines = read_file('../newsbt_data/valid/code_simsbt_camel')
print(len(train_ccode_simsbt_camel_lines)) # 문장개수
print(len(test_code_simsbt_camel_lines)) # 문장개수
print(len(valid_code_simsbt_camel_lines)) # 문장/개수

445812
20000
20000


In [10]:
t4 = Tokenizer(BPE())
t4.pre_tokenizer = Whitespace()
trainer = BpeTrainer(vocab_size=52000,
show_progress = True,
special_tokens = ["<pad>","<unk>", "<start>", "<end>", "<code>"])
t4.train(files=["../newsbt_data/train/code_simsbt_camel"], trainer=trainer)

In [11]:
t4.save("../newsbt_data/vocabulary/code_simsbt_camel_bpe.json")

### indexing newsbtcode

In [12]:
my_tokenizer4 = Tokenizer.from_file("../newsbt_data/vocabulary/code_simsbt_camel_bpe.json")

In [13]:
def make_tokens(tokenizer, lines):
    data = []
    for i in lines:
        output = tokenizer.encode(i)
        data.append(output.ids)
    return data

In [14]:
# out path
train_out_path = '../newsbt_data/train/'
test_out_path = '../newsbt_data/test/'
valid_out_path = '../newsbt_data/valid/'

In [15]:
train_data4 = make_tokens(my_tokenizer4, train_ccode_simsbt_camel_lines)
test_data4 = make_tokens(my_tokenizer4, test_code_simsbt_camel_lines)
valid_data4 = make_tokens(my_tokenizer4, valid_code_simsbt_camel_lines)

save_idx_file(train_out_path, "code_simsbt_camel_bpe", train_data4)
save_idx_file(test_out_path, "code_simsbt_camel_bpe", test_data4)
save_idx_file(valid_out_path, 'code_simsbt_camel_bpe', valid_data4)
